In [1]:
import shorttext
import numpy as np
import shorttext.utils.classification_exceptions as e

from keras.layers import Dense, Reshape
from keras.models import Sequential
from keras.regularizers import l2

Using Theano backend.


In [2]:
subdict = shorttext.data.subjectkeywords()

In [3]:
wvmodel = shorttext.utils.load_word2vec_model('/Users/stephenhky/Data/word2vec/GoogleNews-vectors-negative300.bin')

In [4]:
cnn_caltor = shorttext.classifiers.VarNNEmbeddedVecClassifier(wvmodel)
cnnmodel = shorttext.classifiers.frameworks.CNNWordEmbed(len(subdict))
cnn_caltor.train(subdict, cnnmodel)

Epoch 1/10
45/45 [==============================] - 0s - loss: 1.0449     
Epoch 2/10
45/45 [==============================] - 0s - loss: 0.6358     
Epoch 3/10
45/45 [==============================] - 0s - loss: 0.3522     
Epoch 4/10
45/45 [==============================] - 0s - loss: 0.2135     
Epoch 5/10
45/45 [==============================] - 0s - loss: 0.1463     
Epoch 6/10
45/45 [==============================] - 0s - loss: 0.1070     
Epoch 7/10
45/45 [==============================] - 0s - loss: 0.0869     
Epoch 8/10
45/45 [================

In [5]:
lda128 = shorttext.classifiers.GensimTopicModeler()
lda128.train(subdict, 128)

In [6]:
from sklearn.naive_bayes import GaussianNB

nblda_caltor = shorttext.classifiers.TopicVectorSkLearnClassifier(lda128, GaussianNB())
nblda_caltor.train(subdict)

In [7]:
cnn_caltor.score('linear algebra')

{'mathematics': 0.99993068,
 'physics': 6.0704348e-05,
 'theology': 8.6032642e-06}

In [8]:
nblda_caltor.score('linear algebra')

{'mathematics': 1.0, 'physics': 0.0, 'theology': 0.0}

In [9]:
logit = shorttext.stack.LogisticStackedGeneralization(intermediate_classifiers={'cnn': cnn_caltor, 'nblda': nblda_caltor})
logit.register_classifiers()
logit.register_classlabels(subdict)

In [10]:
logit.train(subdict)

Epoch 1/1000
45/45 [==============================] - 0s - loss: 1.1361     
Epoch 2/1000
45/45 [==============================] - 0s - loss: 1.1330     
Epoch 3/1000
45/45 [==============================] - 0s - loss: 1.1302     
Epoch 4/1000
45/45 [==============================] - 0s - loss: 1.1272     
Epoch 5/1000
45/45 [==============================] - 0s - loss: 1.1243     
Epoch 6/1000
45/45 [==============================] - 0s - loss: 1.1214     
Epoch 7/1000
45/45 [==============================] - 0s - loss: 1.1184     
Epoch 8/1000
45/45 [

In [11]:
logit.score('linear algebra')

{'mathematics': 0.57608134, 'physics': 0.051002707, 'theology': 0.039252926}

In [12]:
logit.score('quantum mechanics')

{'mathematics': 0.03483358, 'physics': 0.43997508, 'theology': 0.028393921}

In [13]:
logit.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
reshape_1 (Reshape)              (None, 6)             0           reshape_input_1[0][0]            
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 3)             21          reshape_1[0][0]                  
Total params: 21
Trainable params: 21
Non-trainable params: 0
____________________________________________________________________________________________________


In [14]:
logit.save_compact_model('/Users/stephenhky/Downloads/tmp1.bin')

In [15]:
logit2 = shorttext.stack.LogisticStackedGeneralization(intermediate_classifiers={'cnn': cnn_caltor, 'nblda': nblda_caltor})

In [16]:
logit2.load_compact_model('/Users/stephenhky/Downloads/tmp1.bin')

In [17]:
logit2.score('quantum mechanics')

{'mathematics': 0.03483358, 'physics': 0.43997508, 'theology': 0.028393921}

In [18]:
logit2.score('linear algebra')

{'mathematics': 0.57608134, 'physics': 0.051002707, 'theology': 0.039252926}